In [ ]:
import warnings

from rasterio.errors import NotGeoreferencedWarning

warnings.simplefilter("ignore", NotGeoreferencedWarning)

In [ ]:
%matplotlib inline

import netCDF4
import numpy as np
import rioxarray
import xarray as xr
from matplotlib import pyplot as plt
from odc.geo.xr import wrap_xr

In [ ]:
xx = xr.open_dataset("Data/sample.nc", chunks={"bands": 5})
xx

In [ ]:
!ls -lh *nc

In [ ]:
#!ncdump -hs Data/sample.nc

In [ ]:
xx["reflectance"]

In [ ]:
xx.to_zarr?

In [ ]:
lon, lat, elev, glt_x, glt_y = [
    xx["location"][n][:] for n in ["lon", "lat", "elev", "glt_x", "glt_y"]
]
wavelength = xx["sensor_band_parameters/wavelengths"][:]

In [ ]:
plt.imshow(elev)
plt.colorbar()

In [ ]:
from odc.geo.math import affine_from_pts, quasi_random_r2
from odc.geo import xy_
from odc.geo.geobox import GeoBox
from odc.geo import geom

ix, iy = quasi_random_r2(100, lon.shape).astype('int32').T
plt.scatter(lon[iy, ix], lat[iy, ix])

In [ ]:
PIX = [xy_(x, y) for x, y in zip(ix, iy)]
WLD = [xy_(x, y) for x, y in zip(lon.data[iy, ix], lat.data[iy, ix])]

gbox = GeoBox(lon.shape, affine_from_pts(PIX, WLD), 4326)

display(gbox)

In [ ]:
wld_pts = geom.multipoint([(pt.x, pt.y) for pt in WLD], 4326)
pix_pts = geom.multipoint([(float(pt.x), float(pt.y)) for pt in PIX], None)

display(wld_pts, wld_pts.to_crs("utm"), pix_pts)


In [ ]:
def gbox_from_points(pix, wld, shape):
    crs = wld.crs
    _pix = [xy_(*g.points[0]) for g in pix.geoms]
    _wld = [xy_(*g.points[0]) for g in wld.geoms]
    A = affine_from_pts(_pix, _wld)
    return GeoBox(shape, A, crs)

gbox_from_points(pix_pts, wld_pts.to_crs("utm"), lon.shape)

In [ ]:
gbox_from_points(pix_pts, wld_pts.to_crs(3857), lon.shape)

In [ ]:
[xy_(*p.points[0]) for p in pix_pts.geoms]

In [ ]:
_p = wld_pts.to_crs("utm")
display(_p.crs, _p)

In [ ]:
ee = wrap_xr(elev.data, gbox)
ee.plot.imshow(size=8)

In [ ]:
ee.odc.geobox.resolution

In [ ]:
ee.odc.write_cog("elev.tif", overwrite=True)

In [ ]:
ee.odc.geobox

In [ ]:
xx.spatial_ref, xx.geotransform[[1, 2, 0, 4, 5, 3]].reshape(2, 3)

In [ ]:
%%time
yy = xx["reflectance"][:, :, :]
yy.shape

In [ ]:
plt.imshow(yy[:, :, 100])

In [ ]:
# plt.imshow(lat); plt.colorbar()

In [ ]:
!ncdump -hs Data/sample.nc

In [ ]:
# xx, *_ = rioxarray.open_rasterio("EMIT_L2A_RFL_001_20230531T133036_2315109_002.nc", chunks=(-1, -1, 1))
# xx.reflectance.data